In [1]:
import pandas as pd

In [2]:
def backtest_signals(data, start_date=None, end_date=None):
    """
    Backtests the trading signals (Buy, Sell, Hold) in the dataset over a specified time frame.

    Args:
        df (pd.DataFrame): DataFrame with stock data including 'Date', 'Close', and 'Signal'.
        start_date (str, optional): Start date in 'YYYY-MM-DD' format. Defaults to the first date in the dataset.
        end_date (str, optional): End date in 'YYYY-MM-DD' format. Defaults to the last date in the dataset.

    Returns:
        dict: Backtest results including total profit, accuracy, and trade log.
    """
    
    df = pd.read_csv(data)
    
    # Convert Date column to datetime for filtering
    df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
    
    # Filter data for the specified time frame
    if start_date:
        df = df[df['Date'] >= pd.to_datetime(start_date)]
    if end_date:
        df = df[df['Date'] <= pd.to_datetime(end_date)]
    
    # Initialize variables
    capital = 100000  # Starting capital
    position = 0  # Number of shares held
    entry_price = 0  # Price at which the position was entered
    trades = []  # To track all trades

    # Iterate through the dataset
    for _, row in df.iterrows():
        if row['Signal'] == 'Buy' and position == 0:
            # Buy shares
            position = capital // row['Close']  # Buy as many shares as possible
            entry_price = row['Close']
            capital -= position * entry_price
            trades.append((row['Date'], 'Buy', entry_price, position))
        
        elif row['Signal'] == 'Sell' and position > 0:
            # Sell shares
            sell_price = row['Close']
            capital += position * sell_price
            profit = (sell_price - entry_price) * position
            trades.append((row['Date'], 'Sell', sell_price, position, profit))
            position = 0  # Reset position
            entry_price = 0
    
    # Final capital including unsold holdings
    if position > 0:
        capital += position * df.iloc[-1]['Close']  # Sell at the last close price
        trades.append((df.iloc[-1]['Date'], 'Final Sell', df.iloc[-1]['Close'], position))
        position = 0

    # Calculate metrics
    total_profit = capital - 100000
    accuracy = sum(1 for trade in trades if len(trade) > 4 and trade[4] > 0) / len(trades) if trades else 0
    results = {
        "Total Profit": total_profit,
        "Accuracy": accuracy * 100,
        "Trade Log": trades,
        "Final Capital": capital
    }
    
    return results

In [3]:
# Example Usage
backtest_results = backtest_signals(
    data='D:\Master_Folder\Data Science Course\Projects\StockMarket\stock_data/NESTLEIND.NS_2024-01-01_to_2024-11-21.csv',  # Your dataset
    start_date='2024-01-01',  # Specify start date
    end_date='2024-11-01'  # Specify end date
)

C:\Users\Arbaz Khan\AppData\Local\Temp\ipykernel_48648\1876049619.py:17: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)


In [4]:
# Display Results
print("Backtest Results:")
print(f"Total Profit: {backtest_results['Total Profit']}")
print(f"Accuracy: {backtest_results['Accuracy']}%")
print(f"Final Capital: {backtest_results['Final Capital']}")
print("Trade Log:")
for trade in backtest_results['Trade Log']:
    print(trade)

Backtest Results:
Total Profit: -5122.39999999998
Accuracy: 0.0%
Final Capital: 94877.60000000002
Trade Log:
(Timestamp('2024-03-13 00:00:00'), 'Buy', 2582.95, 38.0)
(Timestamp('2024-04-18 00:00:00'), 'Sell', 2462.55, 38.0, -4575.199999999986)
(Timestamp('2024-06-21 00:00:00'), 'Buy', 2498.4, 38.0)
(Timestamp('2024-08-01 00:00:00'), 'Sell', 2484.0, 38.0, -547.2000000000035)
